# Decision Tree Algorithm

## Overview
- [1. What is Decision Tree](#1)
- [2. Important Terminology related to Decision Trees](#2)
- [3. How do Decision Trees work?](#3)
- [4. ID3 Algorithm](#4)
    - [4.1 Entropy](#4.1)
    - [4.2 Infomation Gain](#4.2)
    - [4.2 Split a node into branches](#4.3)
- [5. Model from scratch and predict](#5)
- [6. Decision Tree in sklearn and some notes](#6)
    - [6.1 CART cost function for classification](#6.1)
    - [6.2 CART cost function for regression](#6.2)
- [7. Pros and cons](#7)
- [8. References](#8)


### Import package

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

### Data

In [2]:
weather = pd.read_csv('weather.csv')
weather.head()

,outlook,temperature,humidity,wind,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes


<a name='1' ></a>
## 1. What is Decision Tree?
Decision Tree algorithm belongs to the family of supervised learning algorithms. Unlike other supervised learning algorithms, the decision tree algorithm can be used for solving **regression and classification** problems too.

The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

In Decision Trees, for predicting a class label for a record we start from the root of the tree. We compare the values of the root attribute with the record’s attribute. On the basis of comparison, we follow the branch corresponding to that value and jump to the next node.

**Decision tree types:**
- **Classification tree** analysis is when the predicted outcome is the class (discrete) to which the data belongs
- **Regression tree** analysis is when the predicted outcome can be considered a real number (e.g. the price of a house, or a patient's length of stay in a hospital)

<a name='2' ></a>
## 2. Important Terminology related to Decision Trees
- **Root Node**: It represents the entire population or sample and this further gets divided into two or more homogeneous sets
- **Splitting**: It is a process of dividing a node into two or more sub-nodes
- **Decision Node**: When a sub-node splits into further sub-nodes, then it is called the decision node
- **Leaf/Terminal Node**: Nodes do not split is called Leaf or Terminal node
- **Pruning**: When we remove sub-nodes of a decision node, this process is called pruning. You can say the opposite process of splitting
- **Branch/Sub-Tree**: A subsection of the entire tree is called branch or sub-tree
- **Parent and Child Node**: A node, which is divided into sub-nodes is called a parent node of sub-nodes whereas sub-nodes are the child of a parent node

<div style="width:image width px; font-size:80%; text-align:center;"><img src='images/structure-tree.png' alt="alternate text" width="width" height="height" style="width:700px;height:400px;" />  </div>

In [3]:
class Node(object):
    def __init__(
        self, 
        ids: list = None, 
        children: list = [], 
        entropy: float = 0.0, 
        depth: int = 0
    ):
        """
        Represent a node in tree
        
        Args:
            ids: (int) index of data in this node
            children: (list) list of its child nodes
            entropy: (float)
            depth: (int) distance from this node to root node
        """
        
        self.ids = ids  
        self.children = children 
        self.entropy = entropy
        self.depth = depth      
        self.split_attribute = None     # which attribute is chosen, it non-leaf
        self.order = None     # order of values of split_attribute in children
        self.label = None     # label of node if it is a leaf

    def set_properties(self, split_attribute, order):
        self.split_attribute = split_attribute
        self.order = order

    def set_label(self, label):
        self.label = label
        
    def __str__(self):
        information = (
            f'Index of node: {self.ids}\n'
            f'Children: {self.children}\n'
            f'Entropy: {self.entropy}\n'
            f'Depth: {self.depth}'
        )
        return information

<a name='3' ></a>
## 3. How do Decision Trees work?
The decision of making strategic splits heavily affects a tree’s accuracy. The decision criteria are different for classification and regression trees.

Decision trees use multiple algorithms to decide to split a node into two or more sub-nodes. The creation of sub-nodes increases the homogeneity of resultant sub-nodes. In other words, we can say that the purity of the node increases with respect to the target variable. The decision tree splits the nodes on all available variables and then selects the split which results in most homogeneous sub-nodes.

The algorithm selection is also based on the type of target variables. Let us look at some algorithms used in Decision Trees:
- [ID3](https://en.wikipedia.org/wiki/ID3_algorithm) (Iterative Dichotomiser 3)
- [C4.5](https://en.wikipedia.org/wiki/C4.5_algorithm) (successor of ID3)
- [CART](https://en.wikipedia.org/wiki/Predictive_analytics#Classification_and_regression_trees_.28CART.29) (Classification And Regression Tree)
- [Chi-square automatic interaction detection (CHAID)](https://en.wikipedia.org/wiki/Chi-square_automatic_interaction_detection). Performs multi-level splits when computing classification trees
- [MARS](https://en.wikipedia.org/wiki/Multivariate_adaptive_regression_splines): extends decision trees to handle numerical data better

ID3 and CART were invented independently at around the same time (between 1970 and 1980), yet follow a similar approach for learning a decision tree from training tuples. **In this lecture, We firstly concentrate on ID3 Algorithm for DT (Decision Tree).**

<a name='4' ></a>
## 4. ID3 Algorithm 
The ID3 algorithm begins with the original set $S$ as the root node. On each iteration of the algorithm, it iterates through every unused attribute of the set $S$ and calculates the entropy ${H}(S)$ or the information gain $IG(S)$ of that attribute. It then selects the attribute which has the smallest entropy (or largest information gain) value. The set $S$ is then split or partitioned by the selected attribute to produce subsets of the data. (For example, a node can be split into child nodes based upon the subsets of the population whose ages are less than 50, between 50 and 100, and greater than 100.) The algorithm continues to recurse on each subset, considering only attributes never selected before.

**Steps in ID3:**
- Calculate the entropy of every attribute $a$ of the data set $S$.
- Partition ("split") the set $S$ into subsets using the attribute for which the resulting entropy after splitting is minimized; or, equivalently, information gain is maximum.
- Make a decision tree node containing that attribute.
- Recurse on subsets using the remaining attributes.

**Recursion on a subset may stop in one of these cases:**

- every element in the subset belongs to the same class; in which case the node is turned into a leaf node and labelled with the class of the examples.
- there are no more attributes to be selected, but the examples still do not belong to the same class. In this case, the node is made a leaf node and labelled with the most common class of the examples in the subset.
- there are no examples in the subset, which happens when no example in the parent set was found to match a specific value of the selected attribute. An example could be the absence of a person among the population with age over 100 years. Then a leaf node is created and labelled with the most common class of the examples in the parent node's set.

<a name='4.1' ></a>
### 4.1 Entropy

Entropy is a measure of the randomness in the information being processed. The higher the entropy, the harder it is to draw any conclusions from that information. 

In other word, Entropy $H(S)$ is a measure of the amount of uncertainty in the (data) set $S$ (i.e. entropy characterizes the (data) set $S$)

$$H(S) = \sum_{x \in X} -p(x) \log_2 p(x)$$
where, 
- $S$: The current dataset for which entropy is being calculated
- $X$: The set of classes in $S$
- $p(x)$: The proportion of the number of elements in class $x$ to the number of elements in set $S$

When $H(S)=0$, the set $S$ is perfectly classified (i.e. all elements in $S$ are of the same class). In ID3, entropy is calculated for each remaining attribute. The attribute with the smallest entropy is used to split the set $S$ on this iteration. 

In [4]:
def entropy(target):
    """
    Calculate entropy of the label set S
    
    Args:
        target: (ndarray) labels in set S
    Return:
        en: (float) entropy value of input 'target'
    """
    _, freqs = np.unique(target, return_counts=True)
    probs = freqs / float(freqs.sum())
    en = - np.sum(probs * np.log(probs))
    return en
    
# Test function
target = weather['play']
en = entropy(target)
print(f'Target: {target.values} \nEntropy: {en}')

Target: ['no' 'no' 'yes' 'yes' 'yes' 'no' 'yes' 'no' 'yes' 'yes' 'yes' 'yes' 'yes'
 'no'] 
Entropy: 0.6517565611726531


<a name='4.2' ></a>
### 4.2 Information Gain
Information gain $IG(A)$ is the measure of the difference in entropy from before to after the set $S$ is split on an attribute $A$. In other words, how much uncertainty in $S$ was reduced after splitting set $S$ on attribute $A$.

$$IG(S, A) = H(S) - \sum_{t \in T} p(t) H(t) = H(S) - H(S|A)$$
where,
- $H(S)$: Entropy of $S$
- $T$: The subsets created from splitting set $S$ by attribute $A$ such that $S = \cup_{t \in T} t$
- $p(t)$: The proportion of the number of elements in $t$ to the number of elements in set $S$
- $H(t)$: Entropy of subset $t$

In [5]:
def information_gain(node, T, target):
    """
    Calculate information gain of a tree after spliting.
    
    Args:
        node: (Node) (parent) node, node need to be splitted
        T: (list) contain all indexs of each children
        target: (ndarray) labels of set S
    Return:
        ig: information gain of node T after being splitted by attribute A
    """
    n = len(node.ids)
    H_SA = 0
    for ids in T:
        p = len(ids) / n
        H_SA += p * entropy(target[ids])
    ig = node.entropy - H_SA
    
    return ig

<a name='4.3' ></a>
### 4.3 Split a node into branches
In ID3, information gain can be calculated (instead of entropy) for each remaining attribute. The attribute with the **largest** information gain is used to split the set $S$ on this iteration.

In [6]:
def split(node, data, target):
    """
    Function for making best split
    
    Args:
        node: (Node) a node of tree
        data: (DataFrame) original data from input
        target: (ndarray) labels corresponds to each row in data
    Return:
        child_nodes: (list) contain all childrens of parent node
    """
    ids = node.ids 
    best_gain = 0
    min_gain = 1e-4
    min_samples_split = 2
    best_splits = []
    best_attribute = None
    order = None
    sub_data = data.iloc[ids, :]
    
    for i, att in enumerate(attributes):
        values = data.iloc[ids, i].unique().tolist()
        if len(values) == 1: continue     # entropy = 0
        splits = []
        for val in values: 
            sub_ids = sub_data.index[sub_data[att] == val].tolist()
            splits.append(sub_ids)
        # don't split if a node has too small number of points
        if min(map(len, splits)) < min_samples_split: continue
        # information gain
        gain = information_gain(node, splits, target)
        if gain < min_gain: continue     # stop if small gain 
        if gain > best_gain:
            best_gain = gain 
            best_splits = splits
            best_attribute = att
            order = values
    
    node.set_properties(best_attribute, order)
    child_nodes = [Node(ids = split, entropy = entropy(target[split]), depth = node.depth+1) 
                   for split in best_splits]
    
    return child_nodes

In [7]:
# Create root node
data = weather.iloc[:, :-1]
target = weather.iloc[:, -1]
attributes = list(data)
ids = range(len(data))
root = Node(ids=ids, entropy=entropy(target), depth=0)

# Test function
root.children = split(root, data, target)

# Print tree
print('-' * 30 + 'ROOT' + '-' * 30)
print(root)
# Print all children of root
print('-' * 30 + 'Children' + '-' * 30)
for child in root.children:
    print('-' * 50)
    print(child)

------------------------------ROOT------------------------------
Index of node: range(0, 14)
Children: [<__main__.Node object at 0x7f0325c24d00>, <__main__.Node object at 0x7f0325c353d0>, <__main__.Node object at 0x7f0325c35520>]
Entropy: 0.6517565611726531
Depth: 0
------------------------------Children------------------------------
--------------------------------------------------
Index of node: [0, 1, 7, 8, 10]
Children: []
Entropy: 0.6730116670092565
Depth: 1
--------------------------------------------------
Index of node: [2, 6, 11, 12]
Children: []
Entropy: -0.0
Depth: 1
--------------------------------------------------
Index of node: [3, 4, 5, 9, 13]
Children: []
Entropy: 0.6730116670092565
Depth: 1


<a name='5' ></a>
## 5. Model from scratch and predict
As a result, we can calculate entropy and information gain to generate a decision tree from the root node. And here is the Decision Tree (ID3 algorithm) pseudocode.


```
ID3 (Examples, Attributes, Target_Attribute)
    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = +.
    If all examples are negative, Return the single-node tree Root, with label = -.
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the target attribute in the examples.
    Otherwise Begin
        A ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = A.
        For each possible value, vi, of A,
            Add a new tree branch below Root, corresponding to the test A = vi.
            Let Examples(vi) be the subset of examples that have the value vi for A
            If Examples(vi) is empty
                Then below this new branch add a leaf node 
                with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Attributes – {A}, Target_Attribute)
    End
    Return Root
```

Don't worry, the ID3 algorithm was rebuilt from the above pseudocode. If you're curious, you can find more information in the file `utils.py`.

In [8]:
from utils import DecisionTreeID3

X = data
y = target
tree = DecisionTreeID3(max_depth=3, min_samples_split=2)
tree.fit(X, y)

# Test
y_pred = tree.predict(X)
assert (y_pred == y.values).all(), 'Wrong model!'

<a name='6' ></a>
## 6. Decision Tree in `sklearn` and some notes 

*Scikit-Learn uses the CART algorithm, which produces only **binary trees**: nonleaf nodes always have two children (i.e., questions only have yes/no answers). However, other algorithms such as ID3 can produce Decision Trees with nodes that have more than two children.*

`Scikit-Learn` uses the *Classification And Regression Tree (CART)* algorithm to train Decision Trees (also called “growing” trees). The idea is really quite simple: the algorithm first splits the training set in two subsets using a single feature $k$ and a threshold $t_k$. How does it choose $k$ and $t_k$ ? It searches for the pair $(k, t_k)$ that produces the purest subsets (weighted by their size). 

<a name='6.1' ></a>
### 6.1 CART cost function for classification
The cost function that the algorithm tries to minimize is given by 

$$J(k, t_k) = \frac{m_{\text{left}}}{m} G_{\text{left}} + \frac{m_{\text{right}}}{m} G_{\text{right}}$$

where
\begin{cases}
    G_{\text{left/right}} \text{ measures the impurity of the left/right subset} \\
    m_{\text{left/right}} \text{ is the number of instances in the left/right subset}
\end{cases}

and the impurity of subset maybe `gini` (default in `sklearn`) or `entropy`, you can select the entropy impurity measure instead by setting the `criterion` hyperparameter to `entropy`

*Gini impurity is defined by:*
$$G_i = 1 - \sum_{k=1}^{n} p_{i, k}^2$$

where $p_{i, k}$ is the ratio of class $k$ instances among the training instances in the $i^{th}$ node.

**Note: Gini Impurity or Entropy?**

`The truth is, most of the time it does not make a big difference: they lead to similar trees. Gini impurity is slightly faster to compute, so it is a good default. However, when they differ, Gini impurity tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.`

<a name='6.2' ></a>
### 6.2 CART cost function for regression
The CART algorithm works mostly the same way as earlier, except that instead of trying to split the training set in a way that minimizes impurity, it now tries to split the training set in a way that minimizes the $MSE$

$$J(k, t_k) = \frac{m_{\text{left}}}{m} MSE_{\text{left}} + \frac{m_{\text{right}}}{m} MSE_{\text{right}}$$

where
\begin{cases}
    MSE_{\text{node}} = \sum_{i \in \text{node}} (\hat{y}_{\text{node}} - y^{(i)})^2 \text{ measures the impurity of the left/right subset} \\
    \hat{y}_{\text{node}} = \frac{1}{m_{\text{node}}} \sum_{i \in \text{node}} y^{(i)} \\
    m_{\text{left/right}} \text{ is the number of instances in the left/right subset}
\end{cases}

<a name='7' ></a>
## 7. Pros and cons
Although implemented using any algorithm, Decision Tree has the following advantages and disadvantages.

**Some advantages of decision trees are:**
- Simple to understand and to interpret. Trees can be visualized.
- Requires little data preparation. Other techniques often require data normalization, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.
- The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
- Able to handle both numerical and categorical data. However, the scikit-learn implementation does not support categorical variables for now. Other techniques are usually specialized in analyzing datasets that have only one type of variable.
- Able to handle multi-output problems.
- Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.
- Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
- Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.

**The disadvantages of decision trees include:**
- Decision-tree learners can create over-complex trees that do not generalize the data well. This is called overfitting. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
- Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
- Predictions of decision trees are neither smooth nor continuous, but piecewise constant approximations. Therefore, they are not good at extrapolation.
- The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.
- There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
- Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.

<a name='8' ></a>
## 8. References
- Nagesh Singh Chauhan, [Decision Tree Algorithm, Explained](https://www.kdnuggets.com/2020/01/decision-tree-algorithm-explained.html)
- [Decision tree learning](https://en.wikipedia.org/wiki/Decision_tree_learning)
- [ID3 algorithm](https://en.wikipedia.org/wiki/ID3_algorithm)
- sklearn, [Decision Trees](https://scikit-learn.org/stable/modules/tree.html)
- [Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow (Page 177-189)](https://www.knowledgeisle.com/wp-content/uploads/2019/12/2-Aur%C3%A9lien-G%C3%A9ron-Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-Tensorflow_-Concepts-Tools-and-Techniques-to-Build-Intelligent-Systems-O%E2%80%99Reilly-Media-2019.pdf)
- Blog Machine Learning Cơ bản, [Iterative Dichotomiser 3](https://machinelearningcoban.com/2018/01/14/id3/)